# MCP + A2A Walkthrough

This notebook shows the basic MCP and A2A flows for the registry app:

- Discover agent cards via MCP (list + read).
- Call the A2A gateway with JSON-RPC.
- Call the registry HTTP API.
- Placeholder examples for streaming and push notifications.

It assumes `config.yaml` is populated and the registry app is deployed.

In [1]:
from __future__ import annotations

from pathlib import Path
import json

import httpx
import yaml
from databricks.sdk import WorkspaceClient
from fastmcp import Client as MCPClient
from fastmcp.client.transports import StreamableHttpTransport
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message

In [2]:
APP_HOST = "localhost:8000"

Once the app is running, we can get the status of each service.

In [3]:
import requests

if 'localhost' in APP_HOST:
    resp = requests.get(
    f"http://{APP_HOST}/registry/status"
    )
else:
    resp = requests.get(
        f"https://{APP_HOST}/registry/status",
        headers={"Authorization": f"Bearer {token}"},
    )
    
resp.raise_for_status()
print(resp.json())

{'database': {'ok': True, 'error': None}, 'a2a': {'ok': True, 'url': '/a2a'}, 'mcp': {'ok': True, 'url': '/mcp'}, 'test_agent': {'ok': True, 'url': '/test-agent'}}


Let's test the MCP client - we are going to start the servers and then connect to it.

In [4]:
import asyncio
from fastmcp import Client
from fastmcp.client.transports import StreamableHttpTransport

async def main():
    # the app expects a trailing slash
    transport = StreamableHttpTransport("http://localhost:8000/mcp/")
    async with Client(transport) as client:
        tools = await client.list_tools()
        print([t.name for t in tools])
        result = await client.call_tool(
            "list_available_agents",
            {"tags": ["demo"], "limit": 5}
        )
        print(result)

await main()

['list_available_agents', 'invoke_agent']
CallToolResult(content=[TextContent(type='text', text='{"agents":[]}', annotations=None, meta=None)], structured_content={'agents': []}, meta=None, data={'agents': []}, is_error=False)


In [5]:
config_path = Path("config.yaml")
if not config_path.exists():
    config_path = Path("..") / "config.yaml"

config = yaml.safe_load(config_path.read_text(encoding="utf-8")) or {}

if APP_HOST.startswith("http://") or APP_HOST.startswith("https://"):
    REGISTRY_BASE_URL = APP_HOST
else:
    REGISTRY_BASE_URL = f"http://{APP_HOST}"

REGISTRY_BASE_URL = REGISTRY_BASE_URL.rstrip("/")
MCP_BASE_URL = f"{REGISTRY_BASE_URL}/mcp/"
A2A_BASE_URL = f"{REGISTRY_BASE_URL}/a2a/"

assert REGISTRY_BASE_URL.startswith("http"), "REGISTRY_BASE_URL must include http:// or https://"

if "localhost" in APP_HOST:
    headers = {}
else:
    w = WorkspaceClient()
    headers = w.config.authenticate()

print("MCP_BASE_URL:", MCP_BASE_URL)
print("A2A_BASE_URL:", A2A_BASE_URL)

MCP_BASE_URL: http://localhost:8000/mcp/
A2A_BASE_URL: http://localhost:8000/a2a/


## MCP: list and read agent cards

In [6]:
async def list_mcp_resources():
    transport = StreamableHttpTransport(url=MCP_BASE_URL, headers=headers)
    async with MCPClient(transport) as client:
        resources = await client.list_resources()
        return resources

async def read_mcp_resource(uri: str):
    transport = StreamableHttpTransport(url=MCP_BASE_URL, headers=headers)
    async with MCPClient(transport) as client:
        content = await client.read_resource(uri)
        return content

In [7]:
resources = await list_mcp_resources()
print(resources)
if resources:
    content = await read_mcp_resource(resources[0].uri)
    print(content[0].text)

[Resource(name='agent_cards', title=None, uri=AnyUrl('resource://agent_cards'), description=None, mimeType='application/json', size=None, icons=None, annotations=None, meta={'_fastmcp': {'tags': []}})]
{"agents":["test-agent"]}


## A2A: resolve agent card and send message

In [8]:
from uuid import uuid4

async def resolve_agent_card(httpx_client: httpx.AsyncClient):
    base_url = A2A_BASE_URL
    if not base_url.startswith("http"):
        base_url = REGISTRY_BASE_URL
    resolver = A2ACardResolver(httpx_client=httpx_client, base_url=base_url)
    http_kwargs = {"headers": headers} if headers else None
    card = await resolver.get_agent_card(http_kwargs=http_kwargs)
    if isinstance(card.url, str) and card.url.startswith("/"):
        card.url = f"{REGISTRY_BASE_URL}{card.url}"
    return card

async def send_a2a_message(text: str):
    async with httpx.AsyncClient(timeout=60.0, headers=headers or None) as httpx_client:
        card = await resolve_agent_card(httpx_client)
        client_config = ClientConfig(httpx_client=httpx_client)
        factory = ClientFactory(client_config)
        client = factory.create(card)
        message = Message(
            role="user",
            parts=[{"kind": "text", "text": text}],
            message_id=uuid4().hex,
        )
        events = []
        async for event in client.send_message(message):
            events.append(event)
        return events

In [9]:
response = await send_a2a_message('{"action":"list_agents"}')
print(response)

[(Task(artifacts=[Artifact(artifact_id='0cf3e68f-f97c-4699-a8aa-386991894fbe', description=None, extensions=None, metadata=None, name='agents', parts=[Part(root=TextPart(kind='text', metadata=None, text='{\n  "agents": [\n    {\n      "agent_id": "test-agent",\n      "name": "test-agent",\n      "description": "Test agent",\n      "owner": "test",\n      "status": "active",\n      "default_version": "v1",\n      "created_at": "2026-02-09 05:41:57.258317+00:00",\n      "updated_at": "2026-02-09 05:41:57.258317+00:00"\n    }\n  ]\n}'))])], context_id='54df89fa-d3d8-48c0-a509-5394ca21999a', history=[Message(context_id='54df89fa-d3d8-48c0-a509-5394ca21999a', extensions=None, kind='message', message_id='ad49f110cc84488b8a559ee68d012334', metadata=None, parts=[Part(root=TextPart(kind='text', metadata=None, text='{"action":"list_agents"}'))], reference_task_ids=None, role=<Role.user: 'user'>, task_id='7238c82a-b531-4b02-9979-8d771d1a7f42')], id='7238c82a-b531-4b02-9979-8d771d1a7f42', kind='ta

## Registry HTTP API (basic CRUD reads)

In [10]:
def list_agents_http():
    resp = httpx.get(f"{REGISTRY_BASE_URL}/registry/agents", headers=headers, timeout=30.0)
    resp.raise_for_status()
    return resp.json()

def get_agent_http(agent_id: str):
    resp = httpx.get(
        f"{REGISTRY_BASE_URL}/registry/agents/{agent_id}",
        headers=headers,
        timeout=30.0,
    )
    resp.raise_for_status()
    return resp.json()

def get_agent_card_http(agent_id: str, version: str | None = None):
    params = {"version": version} if version else None
    resp = httpx.get(
        f"{REGISTRY_BASE_URL}/registry/agents/{agent_id}/card",
        headers=headers,
        params=params,
        timeout=30.0,
    )
    resp.raise_for_status()
    return resp.json()

In [11]:
agents = list_agents_http()
print(agents)
first_id = agents["agents"][0]["agent_id"]
print(get_agent_http(first_id))
print(get_agent_card_http(first_id))

{'agents': [{'agent_id': 'test-agent', 'name': 'test-agent', 'description': 'Test agent', 'owner': 'test', 'status': 'active', 'default_version': 'v1', 'created_at': '2026-02-09T05:41:57.258317+00:00', 'updated_at': '2026-02-09T05:41:57.258317+00:00'}]}
{'agent_id': 'test-agent', 'name': 'test-agent', 'description': 'Test agent', 'owner': 'test', 'status': 'active', 'default_version': 'v1', 'created_at': '2026-02-09T05:41:57.258317+00:00', 'updated_at': '2026-02-09T05:41:57.258317+00:00'}
{'url': '/a2a', 'name': 'Test Agent', 'skills': [{'id': 'test-agent', 'name': 'Test Agent', 'description': 'pytest'}], 'version': '1.0.0', 'description': 'Test Agent card', 'capabilities': {'streaming': False}, 'defaultInputModes': ['text'], 'defaultOutputModes': ['text']}


## Streaming and push notifications (placeholders)

These APIs require a streaming-capable A2A server and push configuration. The registry gateway currently runs non-streaming.

In [12]:
async def send_a2a_message_streaming(text: str):
    """Placeholder: requires streaming-enabled A2A server."""
    raise NotImplementedError("Streaming requires a streaming-capable A2A server.")

async def configure_push_notifications():
    """Placeholder: requires push notification support in the server."""
    raise NotImplementedError("Push notifications not configured in this app.")